In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [9]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.002339                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.000025
0.000750      2.0 -0.002328                  2.0  0.000013
0.001052      3.0 -0.002302                  3.0  0.000077
0.001476      4.0 -0.002275                  4.0  0.000105
0.002070      5.0 -0.002245                  5.0  0.000134
0.002904      6.0 -0.002213                  6.0  0.000165
0.004074      7.0 -0.002179                  7.0  0.000195
0.005714      8.0 -0.002144                  8.0  0.000228
0.008015      9.0 -0.002110                  9.0  0.000262
0.011243     10.0 -0.002074                 10.0  0.000297
0.015771     11.0 -0.002039                 11.0  0.000333
0.022122     12.0 -0.002006                 12.0  0.000371
0.031031     13.0 -0.001976                 13.0  0.000410
0.043528     14.0 -0.001938                 14.0  0.000457
0.061057     15.0 -0.001706                 15.0  0.000658
0.085645     16.0 -0.001022                 16.0  0.001171
0.120136     17.0  0.000194                 17.0  0.001994
0.168516     18.0  0.001998                 18.0  0.003140
0.236378     19.0  0.004568                 19.0  0.004710
0.331549     20.0  0.008137                 20.0  0.006829
0.465100     21.0  0.013001                 21.0  0.009676
0.652400     22.0  0.018787                 22.0  0.013079
0.915100     23.0  0.020538                 23.0  0.014439
1.283650     24.0  0.017436                 24.0  0.013196
1.800600     25.0  0.013301                 25.0  0.011153
2.525700     26.0  0.009091                 26.0  0.008763
3.542800     27.0  0.005938                 27.0  0.006687
4.969550     28.0  0.003831                 28.0  0.005027
6.970850     29.0  0.002460                 29.0  0.003708
9.778100     30.0  0.001578                 30.0  0.002660
13.715850    31.0  0.001004                 31.0  0.001832
19.239350    32.0  0.000613                 32.0  0.001175
26.987250    33.0  0.000326                 33.0  0.000653
37.855300    34.0  0.000088                 34.0  0.000226
53.100050    35.0 -0.000230                 35.0 -0.000206
73.887500    36.0 -0.000533                 36.0 -0.000565
97.662500    37.0 -0.000686                 37.0 -0.000736
121.437500   38.0 -0.000695                 38.0 -0.000751
145.212500   39.0 -0.000560                 39.0 -0.000625
168.987500   40.0 -0.000357                 40.0 -0.000423
192.762500   41.0 -0.000011                 41.0 -0.000081
216.537500   42.0  0.000976                 42.0  0.000916
240.312500   43.0  0.003050                 43.0  0.002719
264.087500   44.0  0.006380                 44.0  0.005465
287.862500   45.0  0.011093                 45.0  0.010676
311.637500   46.0  0.017297                 46.0  0.017306
335.412500   47.0  0.024992                 47.0  0.023590
359.187500   48.0  0.033631                 48.0  0.031010
382.962500   49.0  0.041574                 49.0  0.039708
406.737500   50.0  0.051361                 50.0  0.049941
430.512500   51.0  0.062585                 51.0  0.059909
454.287500   52.0  0.074948                 52.0  0.070272
478.062500   53.0  0.088141                 53.0  0.082714
501.837500   54.0  0.101695                 54.0  0.097923
525.612500   55.0  0.115445                 55.0  0.114461
549.387500   56.0  0.128890                 56.0  0.129914
573.162500   57.0  0.141863                 57.0  0.142672
596.937500   58.0  0.154412                 58.0  0.152997
620.712500   59.0  0.166588                 59.0  0.162427
644.487500   60.0  0.179091                 60.0  0.173503
668.262500   61.0  0.192112                 61.0  0.187411
692.037500   62.0  0.203990                 62.0  0.202313
715.812500   63.0  0.210877                 63.0  0.213256
739.587500   64.0  0.220012                 64.0  0.227234
76

# Fluxes by Layer

CRD                              CLIRAD  \
                         flug          fldg      fnetg       flug   
pressure     level                                                  
0.000000e+00 1     -16.985901  0.000000e+00 -16.985901        NaN   
1.000000e-08 1            NaN           NaN        NaN -17.306357   
6.244000e-04 2     -16.985901  6.945626e-08 -16.985901 -17.306357   
8.759000e-04 3     -16.985902  9.792714e-08 -16.985901 -17.306357   
1.228600e-03 4     -16.985902  1.393034e-07 -16.985902 -17.306357   
1.723400e-03 5     -16.985902  1.994802e-07 -16.985902 -17.306357   
2.417400e-03 6     -16.985902  2.869742e-07 -16.985902 -17.306357   
3.390900e-03 7     -16.985903  4.141948e-07 -16.985902 -17.306357   
4.756500e-03 8     -16.985903  5.991586e-07 -16.985902 -17.306357   
6.672000e-03 9     -16.985904  8.679899e-07 -16.985903 -17.306357   
9.358900e-03 10    -16.985905  1.258181e-06 -16.985904 -17.306357   
1.312780e-02 11    -16.985906  1.824567e-06 -16.985905 -17.306357   
1.841450e-02 12    -16.985908  2.645982e-06 -16.985906 -17.306357   
2.583020e-02 13    -16.985911  3.835777e-06 -16.985908 -17.306358   
3.623230e-02 14    -16.985916  5.557410e-06 -16.985910 -17.306358   
5.082340e-02 15    -16.985921  8.068197e-06 -16.985913 -17.306359   
7.129060e-02 16    -16.985930  1.216022e-05 -16.985918 -17.306359   
1.000000e-01 17    -16.985941  1.988259e-05 -16.985921 -17.306360   
1.402710e-01 18    -16.985955  3.505121e-05 -16.985920 -17.306358   
1.967600e-01 19    -16.985971  6.448610e-05 -16.985907 -17.306353   
2.759970e-01 20    -16.985985  1.207960e-04 -16.985864 -17.306339   
3.871000e-01 21    -16.985984  2.268441e-04 -16.985757 -17.306307   
5.431000e-01 22    -16.985940  4.234737e-04 -16.985516 -17.306237   
7.617000e-01 23    -16.985800  7.697034e-04 -16.985030 -17.306097   
1.068500e+00 24    -16.985545  1.261390e-03 -16.984284 -17.305877   
1.498800e+00 25    -16.985215  1.820572e-03 -16.983395 -17.305597   
2.102400e+00 26    -16.984835  2.391133e-03 -16.982444 -17.305273   
2.949000e+00 27    -16.984443  2.910659e-03 -16.981532 -17.304930   
4.136600e+00 28    -16.984055  3.358779e-03 -16.980697 -17.304582   
5.802500e+00 29    -16.983680  3.739039e-03 -16.979941 -17.304243   
8.139200e+00 30    -16.983324  4.064373e-03 -16.979260 -17.303930   
1.141700e+01 31    -16.983002  4.355382e-03 -16.978647 -17.303669   
1.601470e+01 32    -16.982737  4.636648e-03 -16.978100 -17.303499   
2.246400e+01 33    -16.982564  4.932747e-03 -16.977632 -17.303475   
3.151050e+01 34    -16.982547  5.264352e-03 -16.977282 -17.303678   
4.420010e+01 35    -16.982786  5.636107e-03 -16.977150 -17.304240   
6.200000e+01 36    -16.983570  5.935569e-03 -16.977634 -17.305426   
8.577500e+01 37    -16.985213  6.076599e-03 -16.979136 -17.307461   
1.095500e+02 38    -16.987273  6.204379e-03 -16.981068 -17.309765   
1.333250e+02 39    -16.989672  6.645990e-03 -16.983026 -17.312289   
1.571000e+02 40    -16.992364  7.760304e-03 -16.984603 -17.315014   
1.808750e+02 41    -16.995481  9.871070e-03 -16.985610 -17.318081   
2.046500e+02 42    -16.999992  1.434941e-02 -16.985642 -17.322452   
2.284250e+02 43    -17.009603  2.670859e-02 -16.982894 -17.331771   
2.522000e+02 44    -17.027356  5.305046e-02 -16.974305 -17.348950   
2.759750e+02 45    -17.056861  1.005240e-01 -16.956337 -17.377380   
2.997500e+02 46    -17.102102  1.770051e-01 -16.925097 -17.420726   
3.235250e+02 47    -17.167252  2.908691e-01 -16.876383 -17.482918   
3.473000e+02 48    -17.256618  4.506199e-01 -16.805998 -17.568304   
3.710750e+02 49    -17.373405  6.621233e-01 -16.711282 -17.680041   
3.948500e+02 50    -17.513677  9.194803e-01 -16.594196 -17.814207   
4.186250e+02 51    -17.678701  1.229153e+00 -16.449547 -17.971969   
4.424000e+02 52    -17.869361  1.596075e+00 -16.273287 -18.154281   
4.661750e+02 53    -18.086167  2.023958e+00 -16.062209 -18.361643   
4.899500e+02 54    -18.329334  2.515359e+00 -15.813975 -18.594432   
5.137250e+02 55    -18.5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')